This file converts an svg file into a gcode format the cable robot can read.  In particular, it converts it to just polylines.

In the future, maybe we can switch to a spline representation, but for now it's polylines.

In [ ]:
import svgpathtools
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
infile = Path('files/brush_path_test_0107_2024.svg')
outfile = infile.with_suffix('.nc')
print(infile, '->', outfile)

In [ ]:
# First reorder the svg
paths, attributes = svgpathtools.svg2paths(infile)
if False:
    new_paths = [paths.pop()]
    while paths:
        end = new_paths[-1].end
        closest = min(paths, key=lambda p: abs(p.start - end))
        new_paths.append(closest)
        paths.remove(closest)

    svgpathtools.disvg(new_paths, filename=infile.with_stem(infile.stem + '_reordered').as_posix(), attributes=attributes)
    paths = new_paths

In [ ]:
def pt_to_mm(pt):
    return pt / 72 * 25.4

In [ ]:
# First convert to numpy polyline
POINT_EVERY_N_MM = 30

polylines = []

for path in paths:
    L = path.length()
    ts = np.arange(0, 1, POINT_EVERY_N_MM / pt_to_mm(L))
    ts = np.append(ts, 1)
    ps = np.array([path.point(t) for t in ts])
    ps = np.conjugate(pt_to_mm(ps))
    polylines.append(np.stack([np.real(ps), np.imag(ps)], axis=1))
    # Sanity check
    # print(np.mean(np.abs(np.diff(ps))), '\t', np.max(np.abs(np.diff(ps[:-1])) - POINT_EVERY_N_MM))
    assert np.all(np.abs(np.diff(ps[:-1])) - POINT_EVERY_N_MM < 0.5)
# Sanity check
for polyline in polylines:
    # print(np.mean(np.linalg.norm(np.diff(polyline, axis=0), axis=1)))
    assert np.all(np.linalg.norm(np.diff(polyline, axis=0), axis=1) - POINT_EVERY_N_MM < 0.5)

In [ ]:
def plot_fig(polylines):
    POINT_EVERY_N_MM = 10
    for polyline in polylines:
        plt.plot(*polyline.T, '--', linewidth=0.5)
    plt.axis('equal')
plot_fig(polylines)

In [ ]:
# Rescale and translate
offset = np.min(np.concatenate(polylines, axis=0), axis=0)
polylines = [polyline - offset for polyline in polylines]
# Separate top and bottom
def condition(polyline):
    return np.all(polyline[:, 1] < 1850)
polylines1 = list(filter(condition, polylines))
polylines2 = list(filter(lambda x: not condition(x), polylines))
# Remove outer rectangle
def condition(polyline):
    return not np.any(np.all(polyline < 5, axis=1), axis=0)
polylines1_inner = list(filter(condition, polylines1))
print(len(polylines1), len(polylines1_inner))
plot_fig(polylines1_inner)

In [ ]:
print(np.min([np.min(polyline, axis=0) for polyline in polylines1], axis=0))
print(np.max([np.max(polyline, axis=0) for polyline in polylines1], axis=0))

In [ ]:
# Export pane 1
outfile1 = outfile.with_stem(outfile.stem + '_pane1')
with open(outfile1, 'w') as f:
    for polyline in polylines1:
        xs, ys = polyline.T
        # Write path to file
        f.write(f'G0 X{xs[0]:.3f} Y{ys[0]:.3f}\n')
        for x, y in zip(xs, ys):
            f.write(f'G1 X{x:.3f} Y{y:.3f}\n')
        f.write(f'G0 X{xs[-1]:.3f} Y{ys[-1]:.3f}\n')
# Export pane 2
outfile2 = outfile.with_stem(outfile.stem + '_pane2')
with open(outfile2, 'w') as f:
    for polyline in polylines2:
        xs, ys = polyline.T
        # Write path to file
        f.write(f'G0 X{xs[0]:.3f} Y{ys[0]:.3f}\n')
        for x, y in zip(xs, ys):
            f.write(f'G1 X{x:.3f} Y{y:.3f}\n')
        f.write(f'G0 X{xs[-1]:.3f} Y{ys[-1]:.3f}\n')

In [ ]:
# Old Export
POINT_EVERY_N_MM = 30

with open(outfile, 'w') as f:
    for path in paths:
        L = path.length()
        # N = int(pt_to_mm(L) / POINT_EVERY_N_MM)
        # ts = np.linspace(0, 1, N)
        ts = np.arange(0, 1, POINT_EVERY_N_MM / pt_to_mm(L))
        ts = np.append(ts, 1)
        ps = np.array([path.point(t) for t in ts])
        ps = pt_to_mm(ps) / 10
        # Sanity check
        print(np.mean(np.abs(np.diff(ps))), '\t', np.max(np.abs(np.diff(ps[:-1] * 10)) - POINT_EVERY_N_MM))
        assert np.all(np.abs(np.diff(ps[:-1] * 10)) - POINT_EVERY_N_MM < 0.5)
        xs, ys = np.real(ps), np.imag(ps)
        # Write path to file
        f.write(f'G0 X{xs[0]:.3f} Y{ys[0]:.3f}\n')
        for x, y in zip(xs, ys):
            f.write(f'G1 X{x:.3f} Y{y:.3f}\n')
        f.write(f'G0 X{xs[-1]:.3f} Y{ys[-1]:.3f}\n')